<a href="https://colab.research.google.com/github/ZahraDehghani99/Natural-Language-Processing/blob/main/HW6/NER_with_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NER using NERA

In [ ]:
!pip install -q NERDA

     |████████████████████████████████| 4.7 MB 49.5 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 596 kB 73.4 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from NERDA.datasets import get_conll_data, download_conll_data

In [ ]:
download_conll_data()

Reading https://data.deepai.org/conll2003.zip


'archive extracted to /root/.conll'

In [ ]:
training = get_conll_data('train')
validation = get_conll_data('valid')

In [ ]:
training.keys()

dict_keys(['sentences', 'tags'])

In [ ]:
training['sentences']

[['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'],
 ['Peter', 'Blackburn'],
 ['BRUSSELS', '1996-08-22'],
 ['The',
  'European',
  'Commission',
  'said',
  'on',
  'Thursday',
  'it',
  'disagreed',
  'with',
  'German',
  'advice',
  'to',
  'consumers',
  'to',
  'shun',
  'British',
  'lamb',
  'until',
  'scientists',
  'determine',
  'whether',
  'mad',
  'cow',
  'disease',
  'can',
  'be',
  'transmitted',
  'to',
  'sheep',
  '.'],
 ['Germany',
  "'s",
  'representative',
  'to',
  'the',
  'European',
  'Union',
  "'s",
  'veterinary',
  'committee',
  'Werner',
  'Zwingmann',
  'said',
  'on',
  'Wednesday',
  'consumers',
  'should',
  'buy',
  'sheepmeat',
  'from',
  'countries',
  'other',
  'than',
  'Britain',
  'until',
  'the',
  'scientific',
  'advice',
  'was',
  'clearer',
  '.'],
 ['We',
  'do',
  "n't",
  'support',
  'any',
  'such',
  'recommendation',
  'because',
  'we',
  'do',
  "n't",
  'see',
  'any',
  'grounds',
  'for',
  '

In [ ]:
print(len(training['sentences']))

14039


In [ ]:
print(len(training['tags']))

14039


In [ ]:
tag_scheme = [
'B-PER',
'I-PER',
'B-ORG',
'I-ORG',
'B-LOC',
'I-LOC',
'B-MISC',
'I-MISC'
]

In [ ]:
transformer = 'bert-base-multilingual-uncased'

In [ ]:
# hyperparameters for network
dropout = 0.1
# hyperparameters for training
training_hyperparameters = {
'epochs' : 2,
'warmup_steps' : 500,                                              
'train_batch_size': 13,                                         
'learning_rate': 0.0001
}

In [ ]:
from NERDA.models import NERDA

model = NERDA(
dataset_training = training,
dataset_validation = validation,
tag_scheme = tag_scheme, 
# tag_outside = 'O',
transformer = transformer,
dropout = dropout,
hyperparameters = training_hyperparameters
)

Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,



 Epoch 1 / 2



 93%|█████████▎| 1003/1080 [05:21<00:25,  3.03it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #13066 length 157 exceeds max_len 128 and has been truncated
  warnings.warn(msg)

100%|██████████| 1080/1080 [05:47<00:00,  3.11it/s]

 67%|██████▋   | 271/407 [00:18<00:09, 14.96it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #2184 length 137 exceeds max_len 128 and has been truncated
  warnings.warn(msg)

100%|██████████| 407/407 [00:27<00:00, 14.71it/s]


Train Loss = 0.2579009973385837 Valid Loss = 0.08403372838155021

 Epoch 2 / 2



 93%|█████████▎| 1003/1080 [05:32<00:25,  3.00it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #13066 length 157 exceeds max_len 128 and has been truncated
  warnings.warn(msg)

100%|██████████| 1080/1080 [05:58<00:00,  3.01it/s]

 67%|██████▋   | 271/407 [00:18<00:09, 15.07it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #2184 length 137 exceeds max_len 128 and has been truncated
  warnings.warn(msg)

100%|██████████| 407/407 [00:27<00:00, 14.80it/s]

Train Loss = 0.06220458389032973 Valid Loss = 0.05815800802196669


'Model trained successfully'

In [ ]:
model.save_network(model_path='model.bin') 

Network written to file model.bin


In [ ]:
test = get_conll_data('test')

In [ ]:
model.evaluate_performance(test)

/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #202 length 139 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 1 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/usr/local/lib/python3.7/dist-packages/NERDA/performance.py:39: UserWarning: length of observed values exceeded lengths of predicted values in 1 cases and were truncated. _Consider_ increasing max_len parameter for your model.
  warnings.warn(f'length of observed values exceeded lengths of predicted values in {n_exceeds} cases and were truncated. _Consider_ increasing max_len parameter for your model.')
/usr/local/lib/pyt

,Level,F1-Score,Precision,Recall
0,B-PER,0.961755,0.974587,0.949257
1,I-PER,0.984017,0.981897,0.986147
2,B-ORG,0.884407,0.884407,0.884407
3,I-ORG,0.860024,0.851939,0.868263
4,B-LOC,0.926026,0.924641,0.927415
5,I-LOC,0.851711,0.829630,0.875000
6,B-MISC,0.817021,0.813559,0.820513
7,I-MISC,0.668151,0.643777,0.694444
0,AVG_MICRO,0.906941,NaN,NaN
0,AVG_MICRO,0.869139,NaN,NaN


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
model.predict_text('Cristiano Ronaldo plays for Juventus FC')

([['Cristiano', 'Ronaldo', 'plays', 'for', 'Juventus', 'FC']],
 [['B-PER', 'I-PER', 'O', 'O', 'B-ORG', 'I-ORG']])

In [ ]:
import torch
torch.save(model, 'model.pt')

In [ ]:
model = torch.load('model.pt') 

# NER using Transformesrs

In [2]:
import pandas as pd
import pickle
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
base_path = "/content/drive/MyDrive/HW6/"

## Prepare Dataset

###Convert text file to dataframe

In this section we want to convert text file to the dataframe in order to do label encoder.

In [ ]:
def convert_file2df(file_name):

  with open(base_path + file_name ,'r') as f:
      lines = f.read().splitlines()

  words, tags = [], []
  for i in range(len(lines)):
    try : 
      x, y = lines[i].split(" ")
      words.append(x)
      tags.append(y)
    except ValueError: # if we have empty line
      pass

  data = {'tokens': words, 'tags': tags}  
  df = pd.DataFrame(data)
  output_file_name = file_name.split(".")[0] 
  df.to_csv(base_path + output_file_name + "_dataframe", index=False)
  return df  

In [ ]:
train_data = convert_file2df("Train.txt")
test_data = convert_file2df("Test.txt")

In [ ]:
train_data.head()

,tokens,tags
0,ميشوند,V
1,.,DELM
2,نتيجهي,N
3,بحث,N
4,بالا,ADJ


In [ ]:
print(f'train_data shape : {train_data.shape}')

train_data shape : (259794, 2)


In [ ]:
train_data.tags.value_counts()

N          109310
P           32580
ADJ         29832
DELM        25844
V           22327
CON         22094
PRO          5714
DET          4130
ADV          3646
QUA          1820
AR           1175
IF            396
SPEC          350
MS            198
MORP          132
PP             85
MQUA           81
PS             31
DEFAULT        20
OH             12
NP             10
OHH             5
INT             2
Name: tags, dtype: int64

In [ ]:
train_tags = [tag for tag, count in train_data.tags.value_counts().items()]
print(f'len train tags : {len(train_tags)}')

len train tags : 23


In [ ]:
test_data.head()

,tokens,tags
0,هجري,ADJ
1,شمسي,ADJ
2,فوت,N
3,كرد,V
4,و,CON


In [ ]:
print(f'test_data shape : {test_data.shape}')

test_data shape : (259794, 2)


In [ ]:
test_data.tags.value_counts()

N          115385
ADJ         32226
P           30790
DELM        27934
CON         20781
V           18826
PRO          4493
DET          3839
ADV          3096
QUA          1418
SPEC          380
IF            177
MORP          176
AR             75
MS             62
PP             56
MQUA           37
DEFAULT        21
OH             10
PS              7
INT             5
Name: tags, dtype: int64

In [ ]:
test_tags = [tag for tag, count in train_data.tags.value_counts().items()]
print(f'len test tags : {len(test_tags)}')

len test tags : 23


#### Label Encoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(train_data["tags"])

print(f'y_train : {y_train}')

y_train : [22  5 12 ... 12 22  5]


let's create a dictionary to map tags to int.

In [ ]:
tag2int = {}
inv =  le.inverse_transform([i for i in range(len(train_tags))])
for i in range(len(inv)):
  tag2int[inv[i]] = i

print(f'tag2int : {tag2int}')

tag2int : {'ADJ': 0, 'ADV': 1, 'AR': 2, 'CON': 3, 'DEFAULT': 4, 'DELM': 5, 'DET': 6, 'IF': 7, 'INT': 8, 'MORP': 9, 'MQUA': 10, 'MS': 11, 'N': 12, 'NP': 13, 'OH': 14, 'OHH': 15, 'P': 16, 'PP': 17, 'PRO': 18, 'PS': 19, 'QUA': 20, 'SPEC': 21, 'V': 22}


In [5]:
def save_dict_in_pkl(dict_name, output_file_name):
  output_file = open(base_path +  output_file_name, 'wb')
  pickle.dump(dict_name, output_file)
  output_file.close() 

def load_pkl(file_name):
  with open(base_path + file_name,'rb') as f:
      file = pickle.load(f)
  return file    

def save_dict_in_json(dict_name, output_file_name):
  with open(base_path + output_file_name, 'w', encoding="utf-8") as fp:
      json.dump(dict_name, fp)

def load_json(file_name):
  with open(base_path + file_name, 'r', encoding="utf-8") as fp:
    data = json.load(fp)
  return data    

In [ ]:
# save_dict_in_pkl(tag2int, "tag2int.pkl")
tag2int = load_pkl("load_pkl")

### Convert text file to dictionary 

برای استفاده از دیتاست باید آن را به فرمت قابل قبول در آوریم 

In [ ]:
def convert_txt2dict(file_name):
  
  '''
  This function read input file and then convert in to the dictionary.
  Producted dictionary have two keys (tokens and tags) :
  tokens contain tokens of each sentence and tags contain tags of each sentence.
  '''
  with open(base_path + file_name ,'r') as f:
      lines = f.read().splitlines()

  #because after each sentence we have blank line, we should append blank
  #line at the end of the last sentence in the file    
  lines.append("") 

  words, tags = [], []
  words_line, tags_lines = [], []
  for i in range(len(lines)):
    try : 
        x, y = lines[i].split(" ")
        words_line.append(x)
        tags_lines.append(y)
    except ValueError: # if we have empty line
        words.append(words_line)
        tags.append(tags_lines)
        words_line, tags_lines= [], []

  data = {'tokens': words, 'ner_tags': tags}   
  output_file_name = file_name.split(".")[0]    
  output_file = open(base_path + output_file_name + ".pkl", 'wb')
  pickle.dump(data, output_file)
  output_file.close()  


تبدیل به فرمت قابل قبول برای transformers

lets convert train.txt and test.txt to dictionary.

In [ ]:
convert_txt2dict("Train.txt")
convert_txt2dict("Test.txt")

Than we load these dictinaries.

In [ ]:
train = load_pkl("Train.pkl")
test = load_pkl("Test.pkl")

In [ ]:
print(f'len of train.pkl : {len(train)}')
print(f'key of train.pkl : {train.keys()}')
print(f'first elements of train["tokens"] :\n{train["tokens"][0]}\n')
print(f'first elements of train["ner_tags"] :\n{train["ner_tags"][0]}')

len of train.pkl : 2
key of train.pkl : dict_keys(['tokens', 'ner_tags'])
first elements of train["tokens"] :
['ميشوند', '.']

first elements of train["ner_tags"] :
['V', 'DELM']


In [ ]:
print(f'number of sentences in train : {len(train["tokens"])}')

tokens = 0
for sent in train["tokens"]:
  tokens += len(sent)

print(f'number of tokens in train :  {tokens}')

number of sentences in train : 8722
number of tokens in train :  259794


In [ ]:
print(f'len of test.pkl : {len(test)}')
print(f'key of test.pkl : {test.keys()}')
print(f'first elements of test["tokens"] :\n{test["tokens"][0]}\n')
print(f'first elements of test["ner_tags"] :\n{test["ner_tags"][0]}')

len of test.pkl : 2
key of test.pkl : dict_keys(['tokens', 'ner_tags'])
first elements of test["tokens"] :
['هجري', 'شمسي', 'فوت', 'كرد', 'و', 'در', 'قم', 'دفن', 'شد', '.']

first elements of test["ner_tags"] :
['ADJ', 'ADJ', 'N', 'V', 'CON', 'P', 'N', 'N', 'V', 'DELM']


In [ ]:
print(f'number of sentences in test : {len(test["tokens"])}')

tokens = 0
for sent in test["tokens"]:
  tokens += len(sent)

print(f'number of tokens in test :  {tokens}')

number of sentences in test : 9280
number of tokens in test :  259794


Let's convert tags into int in train and test dictionaries

In [ ]:
tag2int["N"]

12

In [ ]:
ner_tags_train = []

for sent in train["ner_tags"]:
  sent_tags_int = []
  for token in sent:
    sent_tags_int.append(tag2int[token])
  ner_tags_train.append(sent_tags_int)

In [ ]:
ner_tags_test = []

for sent in test["ner_tags"]:
  sent_tags_int = []
  for token in sent:
    sent_tags_int.append(tag2int[token])
  ner_tags_test.append(sent_tags_int)

In [ ]:
print(f'len ner_tags_train : {len(ner_tags_train)}')
print(f'len ner_tags_test : {len(ner_tags_test)}')

len ner_tags_train : 8722
len ner_tags_test : 9280


In [ ]:
train_data = {"tokens": train["tokens"], "ner_tags": ner_tags_train}
test_data = {"tokens": test["tokens"], "ner_tags": ner_tags_test}

In [6]:
# save_dict_in_pkl(train_data, "Train_modified.pkl")
# save_dict_in_pkl(test_data, "Test_modified.pkl")
train_data = load_pkl("Train_modified.pkl")
test_data = load_pkl("Test_modified.pkl") 

## Create dataset in the DatasetDict format

In [1]:
!pip install -q datasets

In [39]:
df_train = pd.DataFrame(train_data)
df_train.head()

,tokens,ner_tags
0,"[ميشوند, .]","[22, 5]"
1,"[نتيجهي, بحث, بالا, اين, است, كه, فعلها, از, ن...","[12, 12, 0, 18, 22, 3, 12, 16, 12, 12, 16, 12,..."
2,"[صورتهاي, گسترشيافته, و, نيز, الگوي, برجستگي, ...","[12, 0, 3, 3, 12, 12, 20, 12, 16, 12, 12, 16, ..."
3,"[هر, كدام, از, فعلهاي, سهگانه, بالا, در, صورت,...","[20, 12, 16, 12, 0, 0, 16, 12, 12, 12, 0, 22, ..."
4,"[در, زير, به, الگوي, برجستگي, گروههاي, فعلي, م...","[16, 12, 16, 12, 12, 12, 0, 22, 3, 12, 18, 0, ..."


In [40]:
df_test = pd.DataFrame(test_data)
df_test.head()

,tokens,ner_tags
0,"[هجري, شمسي, فوت, كرد, و, در, قم, دفن, شد, .]","[0, 0, 12, 22, 3, 16, 12, 12, 22, 5]"
1,"[پروين, اعتصامي, ،, از, داستان, "", ويلانالدوله...","[12, 12, 5, 16, 12, 5, 12, 5, 12, 5, 18, 22, 1..."
2,"[محض, خالي, نبودن, عريضه, با, چايي, مقدار, معت...","[16, 0, 12, 12, 16, 12, 12, 0, 12, 0, 0, 22, 5..."
3,"[بعد, معلوم, ميشود, وقتيكه, ويلانالدوله, خواب,...","[3, 0, 22, 3, 12, 12, 22, 5, 12, 16, 12, 5, 12..."
4,"[ويلانالدوله, خدا, را, شكر, ميكند, كه, آخرش, پ...","[12, 12, 16, 12, 22, 3, 12, 12, 16, 12, 12, 3,..."


In [41]:
import datasets
from datasets import DatasetDict

dataset = DatasetDict()

In [42]:
dataset

DatasetDict({
    
})

https://discuss.huggingface.co/t/convert-a-list-of-dictionaries-to-hugging-face-dataset-object/14670

In [43]:
train = datasets.Dataset.from_pandas(pd.DataFrame(df_train))
test = datasets.Dataset.from_pandas(pd.DataFrame(df_test))

dataset['train'] = train
dataset['test'] = test

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8722
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 9280
    })
})

In [47]:
dataset['train'][0]

{'ner_tags': [22, 5], 'tokens': ['ميشوند', '.']}